In [ ]:
import pandas as pd
import numpy as np
from scipy.linalg import lu_factor, lu_solve
from scipy.optimize import fsolve
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from tqdm import tqdm
from src.utils import is_divisible_by, assert_is_positive, display_df
import src.config as config

# (a)

In [ ]:
def c_stat(y: NDArray[np.float64]) -> NDArray[np.float64]:
    return np.cosh(4 * y) / np.cosh(4)

y = np.linspace(0, 1, 100)

plt.plot(y, c_stat(y), label='$c_{\\text{stat}}(y) = \\frac{\\cosh(4y)}{\\cosh(4)}$')
plt.axhline(0.1, color='r', linestyle='--', label='$c(y) = 0.1$')
plt.xlabel('$y$')
plt.ylabel('$c(y)$')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(config.FIGURES_DIR / 'ex_a_stationary_solution.svg')
plt.show()

In [ ]:
y_solution = fsolve(lambda y: c_stat(y) - 0.1, 0.4)
y_solution[0]

# (g)

In [ ]:
def create_matrix_A(*, n: int, v: float, dy: float, beta: float):
    factor1 = -1 / (v * (dy**2))
    factor2 = -beta**2 / v

    main_diag = 2 * np.ones(n + 1)
    off_diag = -1 * np.ones(n)
    
    matrix1 = np.diag(main_diag) + np.diag(off_diag, k=1) + np.diag(off_diag, k=-1)
    matrix1[0, 1] = -2 # Neumann boundary condition at y=0
    matrix2 = np.eye(n + 1)

    A = factor1 * matrix1 + factor2 * matrix2
    return A

def create_r(*, n: int, dy: float):
    r = np.zeros(n + 1) # r is a constant as it does not depend on x
    r[-1] = 1 / dy**2 # Dirichlet boundary condition at y=1
    return r

def create_u_0(*, n: int):
    '''
    Creates an initial concentration array `u_0` for x=0 as defined by the 
    Dirichlet boundary condition at x=0.

    The array `u_0` represents the concentration values at x=0 as a function of y.
    It includes the concentrations at y_0, y_1, ..., y_n but omits y_{n+1}.

    So, `u_0` = [c(0, y_0), c(0, y_1), ..., c(0, y_n)].
    '''
    return np.zeros(n + 1)

def get_alpha(*, w: NDArray[np.float64], dy: float):
    return (w[-3] - 4 * w[-2] + 3 * w[-1]) / (2 * dy)

In [ ]:
v = 1
beta = 4
dy = 0.05
dx = 0.02 # tau
theta = 3/4
x_lst = dx * np.array([5, 10, 15, 20, 25])

# Verify constants
assert_is_positive(v, 'v')
assert_is_positive(dy, 'dy')
assert is_divisible_by(1, dy), 'dy must divide 1.'
assert_is_positive(dx, 'dx')
assert 0 <= theta <= 1, 'theta must be in [0,1].'
assert all(is_divisible_by(x, dx) for x in x_lst), 'All x values must be divisible by dx.'
assert all(x >= 0 for x in x_lst), 'All x values must be nonnegative.'

N = int(1 / dy) - 1
num_steps = int(sorted(x_lst)[-1] / dx)
y = np.linspace(0, 1, N + 2, dtype=np.float64)

In [ ]:
A = create_matrix_A(n=N, v=v, dy=dy, beta=beta)
r = create_r(n=N, dy=dy)

I = np.eye(N + 1)
matrix = I - theta * dx * A
lu, piv = lu_factor(matrix)
def theta_method(
    *,
    u_n: NDArray[np.float64],
    A: NDArray[np.float64],
    r: NDArray[np.float64],
    dx: float,
    theta: float,
) -> NDArray[np.float64]:
    b = u_n + dx * (1 - theta) * (A @ u_n + r) + dx * theta * r
    u_next = lu_solve((lu, piv), b)
    return u_next

u = np.empty(
    shape=num_steps + 1, # +1 to include u_0
    dtype=[('x', np.float64), ('c_y', np.float64, (N + 2,))]
)
u_0_numerical = create_u_0(n=N)
u_0_full = np.append(u_0_numerical, 1) # Append 1 to include c(0, 1) = 1 Dirichlet boundary condition
u[0] = (0, u_0_full)
c_y_numerical = u_0_numerical

for n in tqdm(range(num_steps)):
    x = (n + 1) * dx
    c_y_numerical = theta_method(u_n=c_y_numerical, A=A, r=r, dx=dx, theta=theta)
    c_y_full = np.append(c_y_numerical, 1)
    u[n + 1] = (x, c_y_full)

for x in x_lst:
    c_y = u[u['x'] == x]['c_y'][0]
    plt.plot(y, c_y, label=f'$x = {x:.2f}$')

plt.xlabel('$y$')
plt.ylabel('$c(y)$')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(config.FIGURES_DIR / 'ex_g_numerical_solution_c.svg')
plt.show()

In [ ]:
alpha_values = pd.DataFrame({'n': [5, 10, 15, 20, 25], 'alpha': [get_alpha(w=u[n]['c_y'], dy=dy) for n in [5, 10, 15, 20, 25]]})
display_df(alpha_values)

In [ ]:
4 * np.sinh(4 * 1) / np.cosh(4)

# (h)

In [ ]:
v = 1
beta = 4
dy = 0.05
dx = 0.02 # tau
theta = 1/2
x_lst = dx * np.array([5, 10, 15, 20, 25])

# Verify constants
assert_is_positive(v, 'v')
assert_is_positive(dy, 'dy')
assert is_divisible_by(1, dy), 'dy must divide 1.'
assert_is_positive(dx, 'dx')
assert 0 <= theta <= 1, 'theta must be in [0,1].'
assert all(is_divisible_by(x, dx) for x in x_lst), 'All x values must be divisible by dx.'
assert all(x >= 0 for x in x_lst), 'All x values must be nonnegative.'

N = int(1 / dy) - 1
num_steps = int(sorted(x_lst)[-1] / dx)
y = np.linspace(0, 1, N + 2, dtype=np.float64)

In [ ]:
A = create_matrix_A(n=N, v=v, dy=dy, beta=beta)
r = create_r(n=N, dy=dy)

I = np.eye(N + 1)
matrix = I - theta * dx * A
lu, piv = lu_factor(matrix)
def theta_method(
    *,
    u_n: NDArray[np.float64],
    A: NDArray[np.float64],
    r: NDArray[np.float64],
    dx: float,
    theta: float,
) -> NDArray[np.float64]:
    b = u_n + dx * (1 - theta) * (A @ u_n + r) + dx * theta * r
    u_next = lu_solve((lu, piv), b)
    return u_next

u = np.empty(
    shape=num_steps + 1, # +1 to include u_0
    dtype=[('x', np.float64), ('c_y', np.float64, (N + 2,))]
)
u_0_numerical = create_u_0(n=N)
u_0_full = np.append(u_0_numerical, 1) # Append 1 to include c(0, 1) = 1 Dirichlet boundary condition
u[0] = (0, u_0_full)
c_y_numerical = u_0_numerical

for n in tqdm(range(num_steps)):
    x = (n + 1) * dx
    c_y_numerical = theta_method(u_n=c_y_numerical, A=A, r=r, dx=dx, theta=theta)
    c_y_full = np.append(c_y_numerical, 1)
    u[n + 1] = (x, c_y_full)

for x in x_lst:
    c_y = u[u['x'] == x]['c_y'][0]
    plt.plot(y, c_y, label=f'$x = {x:.2f}$')

plt.xlabel('$y$')
plt.ylabel('$c(y)$')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(config.FIGURES_DIR / 'ex_h_numerical_solution_c.svg')
plt.show()

In [ ]:
alpha_values = pd.DataFrame({'n': [5, 10, 15, 20, 25], 'alpha': [get_alpha(w=u[n]['c_y'], dy=dy) for n in [5, 10, 15, 20, 25]]})
display_df(alpha_values)